# Forest future

In this notebook I will conclude whether world's forests are safe from disappearing or not. I will focus on future trends
and those areas, which I consider as the most important ones.


In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import statsmodels as stat 
import warnings
from ipywidgets import interact, widgets

warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
forest_data = pd.read_csv('Forest_Area.csv')

forest_data.head()

,CountryID,Country and Area,"Forest Area, 1990","Forest Area, 2000","Forest Area, 2010","Forest Area, 2015","Forest Area, 2020","Total Land Area, 2020","Forest Area as a Proportion of Total Land Area, 2020","Deforestation, 2015-2020","Total Forest Area Affected by Fire, 2015"
0,1.0,WORLD,4236433.42,4158049.52,4106316.94,...,4058930.81,13003000.0,31.13,10200,98040
1,4.0,Afghanistan,1208.44,1208.44,1208.44,1208.44,1208.44,65286.0,1.85,...,...
2,8.0,Albania,788.80,769.30,782.07,789.19,788.90,2740.0,28.79,...,...
3,12.0,Algeria,1667.00,1579.00,1918.00,1956,1949.00,238174.0,0.82,5.4,...
4,16.0,American Samoa,18.07,17.73,17.43,17.28,17.13,20.0,85.65,...,0


We have got 233 different countries with data collected for the whole planet. Now it's time to replace the missing data and preprocess it to make it clearer for further analysis

In [3]:
forest_data.head()
forest_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 11 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   CountryID                                             237 non-null    float64
 1   Country and Area                                      237 non-null    object 
 2   Forest Area, 1990                                     237 non-null    float64
 3   Forest Area, 2000                                     237 non-null    float64
 4   Forest Area, 2010                                     237 non-null    float64
 5   Forest Area, 2015                                     237 non-null    object 
 6   Forest Area, 2020                                     237 non-null    float64
 7   Total Land Area, 2020                                 237 non-null    float64
 8   Forest Area as a Proportion of Total Land Area, 2020  237 no

In [4]:
forest_data['Forest Area, 2015'].fillna(forest_data['Forest Area, 2010'],  inplace = True)
for col in forest_data.columns[2:7]:
    forest_data[col] = pd.to_numeric(forest_data[col], errors='coerce')

forest_data.info()
#to check whether our values are really float or not
print(forest_data['Forest Area, 2015'].unique())
print(forest_data['Forest Area, 2015'].dtype)

for col in forest_data.columns[2:]:  
    forest_data[col] = pd.to_numeric(forest_data[col], errors='coerce')

# Obsłuż wartości NaN, np. zastąp średnią lub medianą, lub usuń wiersze zawierające NaN
forest_data.fillna(method='ffill', inplace=True) 
forest_data.replace([np.inf, -np.inf], np.nan, inplace=True)
forest_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 11 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   CountryID                                             237 non-null    float64
 1   Country and Area                                      237 non-null    object 
 2   Forest Area, 1990                                     237 non-null    float64
 3   Forest Area, 2000                                     237 non-null    float64
 4   Forest Area, 2010                                     237 non-null    float64
 5   Forest Area, 2015                                     236 non-null    float64
 6   Forest Area, 2020                                     237 non-null    float64
 7   Total Land Area, 2020                                 237 non-null    float64
 8   Forest Area as a Proportion of Total Land Area, 2020  237 no

# Forestation future trends 
Now, after cleaning the data, let's show forestation trend over the countries

In [5]:
trend_colors = []

years = np.array([1990, 2000, 2010, 2015, 2020]).reshape(-1, 1)
world_forest_data = forest_data[forest_data['Country and Area'] == 'WORLD']
countries_forest_data = forest_data[forest_data['Country and Area'] != 'WORLD']

# Function to plot trend in afforestation in chosen country
def plot_country_forest_area(country):
    plt.figure(figsize=(10, 6))
    for index, row in forest_data.iterrows():
        if row['Country and Area'] == country:
            forest_area = pd.to_numeric(row[2:7], errors='coerce').values.reshape(-1, 1)
            model = np.polyfit(years.flatten(), forest_area.flatten(), 1)
            slope = model[0]  
            if slope < -0.1:
                trend_color = "red"
            elif -0.1 <= slope <= 0.1:
                trend_color = "grey"
            else:
                trend_color = "green"
            plt.plot(years.flatten(), row[2:7], label=row['Country and Area'], color=trend_color)
    plt.xlabel("Year")
    plt.ylabel("Forest Area (ha)")
    plt.title(f"Forest Area Change Over Time - {country}")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()

# User's list with countries
country_dropdown = widgets.Dropdown(
    options=forest_data['Country and Area'].unique(),
    description='Country:',
    disabled=False,
)

# Interactive box to choose country
interact(plot_country_forest_area, country=country_dropdown)

<Figure size 1000x600 with 0 Axes>

interactive(children=(Dropdown(description='Country:', options=('WORLD', 'Afghanistan', 'Albania', 'Algeria', …

<function __main__.plot_country_forest_area(country)>

Overall trend is negative, but in smaller countries it is probably reversed. Let's see if I'm right.